In [25]:
from nltk import word_tokenize
from collections import defaultdict
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import io
import numpy as np
import nltk


KeyboardInterrupt: 

In [3]:
df = pd.read_csv('o365Data.csv', dtype=object)

In [4]:
all_subjects = df['EmailSubject'].tolist()

In [5]:
receipt_subjects = [s.lower() for s in all_subjects if isinstance(s,str)]

In [6]:
import mailbox

In [7]:
mbox = mailbox.mbox('emails.mbox')

In [8]:
non_receipt_emails = []
non_receipt_subjects = []
for message in mbox:
    payload = message.get_payload(decode=False)
    non_receipt_subjects.append(message['Subject'])
    if isinstance(payload,list):
        non_receipt_emails.append(payload[0])
    else:
        non_receipt_emails.append(payload) 


In [9]:
def clean_gmail_subjects(subs): 
    subs = [s for s in subs if isinstance(s,str)]
    
    remove_words = ['invoice','utf','utf-8','receipt','amazon','paypal','payment','confirmation','hughes','trip','booking','lance','weston','order','reservation','confirmed','itinerary','statement','folio','bill']

    return[s.lower() for s in subs if not any(word.lower() in s.lower() for word in remove_words)]

In [10]:
non_receipt_subjects = clean_gmail_subjects(non_receipt_subjects)

In [11]:
import random

def get_test_data_labels(zeros_list,ones_list):
    labels = np.zeros(len(zeros_list)).tolist() + np.ones(len(ones_list)).tolist()
    
    all_data = zeros_list + ones_list
    combined = list(zip(all_data, labels))
    random.shuffle(combined)

    all_data[:], labels[:] = zip(*combined)
    return all_data,labels

In [12]:
subjects, labels = get_test_data_labels(non_receipt_subjects,receipt_subjects)

In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
X_test = []
y_test = []
predicted = []

def classify(data, labels):
    global X_test,y_test,predicted
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
    vectorizer = CountVectorizer()
#     clf = MultinomialNB()
    clf = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=7, tol=None)
    pipe = Pipeline([('vect', vectorizer),
                        ('tfidf', TfidfTransformer()),
                        ('clf', clf)
                        ])
                        
    pipe.fit(X_train, y_train)  
    predicted = pipe.predict(X_test)
    
    return np.mean(predicted == y_test), clf, vectorizer,pipe

In [19]:
accuracy, clf, vectorizer,pipe = classify(subjects,labels)
print(accuracy)


0.9753255654557916


In [15]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

        0.0       0.97      1.00      0.98      6960
        1.0       0.99      0.93      0.96      3253

avg / total       0.98      0.98      0.98     10213



In [16]:
z = [X_test[i] for i in range(len(X_test)) if y_test[i] == 1 and predicted[i] == 0]
z


['tv licence application for executive assistant',
 'cost of taxi',
 'bijgaand onze factuur:1129638       :kolsteeg/michael mr                :np1/wl',
 'fwd: july at&t',
 'thanks for subscribing',
 're: mssp sow',
 '0100002226 rb65wk weidmann/benjaminmarkusmr 05/09/2018',
 'salesforce productivity and automation has landed to your inbox',
 'ultimate air shuttle #270359 luk-mmu 9/24/2018',
 "we've delivered your purchase",
 'hilton dallas',
 'onds meeting',
 're: concur sales training & happy hour',
 'thanks for buying your tv licence',
 'american expres',
 'you have been refunded by the ucl online store',
 'rec',
 '7/27/18 internet ',
 'itin/inv for andrew.j heinen - travel date june 27, 2018',
 're: adl b warehouse roof fix (reminder that meeting is on tue. april 18th @ 9:30 am)',
 're: texting feauture',
 'uberx',
 're: microsoft (outlook, teams) follow up ',
 '[external] your earlybird check-in refund',
 'hipchat',
 'lunch aaron slydell',
 're: sap 4296 - powder springs',
 'fw: [コン

In [20]:
body = "your republic parking invoice"

pipe.predict([body])


array([1.])

In [21]:
def is_receipt(subject):
    return pipe.predict([subject])[0] == 1

In [23]:
is_receipt("yes")

False

In [18]:
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [19]:
show_most_informative_features(vectorizer,clf,1000)

	-4.2366	voice          		13.8470	receipt        
	-4.0222	intellisense   		13.1045	order          
	-3.7778	ecco           		10.5558	confirmation   
	-3.4325	newcomer       		10.0706	invoice        
	-3.0411	game           		9.6156	reservation    
	-2.7605	tonight        		8.4209	payment        
	-2.3773	ideas          		8.0158	trip           
	-2.2845	in             		6.8297	ride           
	-2.2332	just           		6.1436	itinerary      
	-2.2016	off            		6.0372	booking        
	-1.9693	history        		5.9784	statement      
	-1.8446	school         		5.2617	folio          
	-1.7834	news           		5.2210	bill           
	-1.7713	rate           		4.5663	stripe         
	-1.6794	level          		4.4040	bills          
	-1.6749	echo           		4.1160	national       
	-1.6717	fabrication    		4.1049	enterprise     
	-1.6428	recap          		3.9946	ref            
	-1.5826	meetup         		3.9009	outlook        
	-1.4979	tap            		3.8822	uber           
	-1.4603	week   

	0.0000	202581291      		0.0000	validating     
	0.0000	2028           		0.0000	valeta         
	0.0000	2042451497     		0.0000	valet          
	0.0000	20467          		0.0000	valentine      
	0.0000	2049           		0.0000	vaienh         
	0.0000	2055339558     		0.0000	vader          
	0.0000	207            		0.0000	vadcon         
	0.0000	207265767      		0.0000	vacations      
	0.0000	2074           		0.0000	vacationed     
	0.0000	20aug          		0.0000	vacation       
	0.0000	20aug18        		0.0000	va             
	0.0000	20feb17        		0.0000	v8             
	0.0000	20jul18        		0.0000	v6             
	0.0000	20jun          		0.0000	v5             
	0.0000	20jun17        		0.0000	v4             
	0.0000	20mar17        		0.0000	v3x            
	0.0000	20may17        		0.0000	v3             
	0.0000	20may18        		0.0000	v2x            
	0.0000	20oct18        		0.0000	v2             
	0.0000	20sep18        		0.0000	v0             
	0.0000	20th           		0.0000	uzoom   

In [20]:
import eli5
eli5.show_weights(clf, top=10)

Weight?,Feature
+13.847,x10400
+13.105,x9369
+10.556,x4215
+10.071,x7139
+9.616,x10597
+8.421,x9619
+8.016,x12546
+6.830,x10696
+6.144,x7218
+6.037,x3318
